In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
class write():    
    def printExamples(self):
        grouped = self.df.groupby(['organization name', 'website'])
        for (org_name, website), group_data in grouped:
            print("***************************")
            print(f"Organization: {org_name}, Website: {website}")
            for col in ['generated_description', 'Tasks/Jobs', 'Industry', 'Customers', 'generated_description_conf_interval', 'parsed_description_conf_interval']:
                print(f"{col}: {group_data.iloc[0][col]}")
            for example, example_data in group_data.groupby('example'):
                print("_________________________")
                print(f"Example: {example}")
                for col in ['situation_conf_interval', 'situation_conf_interval_reasoning']:
                    print(f"{col}: {example_data.iloc[0][col]}")
                for idx, row in example_data.iterrows():
                    print("##########################")
                    for col in ['job', 'onet_title', 'onet_task', 'example_job_title', 'task_similarity', 'job_title_similarity', 'onet_weight']:
                        print(f"{col}: {row[col]}")
            print("***************************\n")
        
        def writeExamples(self):
            output_file_path = "../output/examples.txt"
            with open(output_file_path, 'w') as f:
                grouped = self.df.groupby(['organization name', 'website'])
                for (org_name, website), group_data in grouped:
                    f.write("***************************\n")
                    f.write(f"Organization: {org_name}, Website: {website}\n")
                    for col in ['generated_description', 'Tasks/Jobs', 'Industry', 'Customers', 'generated_description_conf_interval', 'parsed_description_conf_interval']:
                        f.write(f"{col}: {group_data.iloc[0][col]}\n")
                    for example, example_data in group_data.groupby('example'):
                        f.write("_________________________\n")
                        f.write(f"Example: {example}\n")
                        for col in ['situation_conf_interval', 'situation_conf_interval_reasoning']:
                            f.write(f"{col}: {example_data.iloc[0][col]}\n")
                        for idx, row in example_data.iterrows():
                            f.write("##########################\n")
                            for col in ['job', 'onet_title', 'onet_task', 'example_job_title', 'task_similarity', 'job_title_similarity','onet_weight']:
                                f.write(f"{col}: {row[col]}\n")
                    f.write("***************************\n\n")
                        
        def generateOutput(self):
            self.conf_interval_text = self.conf_interval if self.conf_interval != True else 'all'
            output_file_path = f"../output/output_{self.task_sim}_{self.title_sim}_{self.conf_interval_text}.txt"
            
            with open(output_file_path, 'w') as f:
                
                group_sums = self.df.groupby('Minor Group Name')['onet_weight'].sum().sort_values(ascending=False)
                f.write(f"{self.num_onet_startups} Startups Founded post launch of chatGPT with 1-10 employees\n")
                
                f.write(f"Task Similarity: {self.task_sim}, Title Similarity {self.title_sim}, Confidence Interval: {self.conf_interval_text}\n")
                f.write(f"{self.num_onet_startups} startups out of {self.num_startups} decomposed\n")
                f.write("Decomposition of startup effect on labor market\n")
                
                for group_name in group_sums.index:
                    group_data = self.df[self.df['Minor Group Name'] == group_name]
                    f.write("*****************\n")
                    f.write(f"Group: {group_name}\n")
                    onet_titles = group_data.groupby('onet_title')['onet_weight'].sum().round(2).sort_values(ascending=False).reset_index()
                    onet_tasks = group_data.groupby('onet_task')['onet_weight'].sum().round(2).sort_values(ascending=False).reset_index()
                    
                    f.write("________________\n")
                    f.write("10 Most Highly Weighted Titles:\n")
                    for index, row in onet_titles.iterrows():
                        f.write(f"Title: {row['onet_title']}, Weight: {row['onet_weight']}\n")
                    f.write("\n")
                
                    f.write("10 Most Highly Weighted Tasks:\n")
                    for index, row in list(onet_tasks.iterrows())[:10]:
                        f.write(f"Task: {row['onet_task']}, Weight: {row['onet_weight']}\n")
                    f.write("\n")
                
                    f.write("10 Example Startups:\n")
                    startups = group_data.groupby(['organization_name','website'])['onet_weight'].sum().round(2).sort_values(ascending=False).reset_index()
                    for index, row in list(startups.iterrows())[:10]:
                        f.write(f"Startup: {row['organization_name']}, Website: {row['website']}\n")
                    f.write("\n")
        


In [15]:
class output():
    def __init__(self):
        self.codes = pd.read_csv("../input/soc_codes/soc_codes.csv",index_col=0)

    
    def getONET(self,task_sim, title_sim, conf_interval=True):
        self.task_sim = task_sim
        self.title_sim = title_sim
        self.conf_interval = conf_interval
        self.onet_df = pd.read_csv("../output/results v3/onet_df.csv")
        self.onet_df = self.onet_df[(self.onet_df.task_similarity > task_sim) & (self.onet_df.job_title_similarity > title_sim)].reset_index(drop=True)
        self.onet_df = self.onet_df[self.onet_df.situation_conf_interval > conf_interval]
        self.computeONETWeights()
        self.getStartupData()
        
        print(f"{len(self.onet_df)} tasks with task similarity > {task_sim} and title similarity > {title_sim}")
        self.num_onet_startups = len(self.df["organization name"].unique())
        self.num_startups = len(self.startup_df["organization name"].unique())
        print(f"{self.num_onet_startups} startups out of {self.num_startups} decomposed")
        
    def computeONETWeights(self):
        self.onet_weights = self.onet_df.groupby("organization_name")["onet_title"].count().apply(lambda x: 1/x).reset_index().rename({"onet_title":"onet_weight"},axis=1)
        self.onet_df = self.onet_df.merge(self.onet_weights, on="organization_name")

    def getONETTitles(self):
        self.onet_titles = self.df.groupby("onet_title").aggregate({"onet_weight":"sum","Detailed Occupation":"first"})
        self.onet_titles.onet_weight = self.onet_titles.onet_weight.round(2)
        self.onet_titles = self.onet_titles.reset_index().sort_values(by="onet_weight", ascending=False) 
        return self.onet_titles

    def getStartupGeneratedLLMTitles(self):
        self.example_job_titles_df = onet_df.example_job_title.value_counts().apply(lambda x: x/3).reset_index()
        return self.example_job_titles_df

    def printTop100ONETTasks(self):
        self.top_100_ONETtasks = onet_df['onet_task'].value_counts().reset_index().head(100)
        self.top_100_tasks.columns = ['Task', 'Frequency']
        for index, row in self.top_100_tasks.iterrows():
            print(f"Task = {row['Task']}, Frequency = {row['Frequency']}\n")
    
    def getStartupData(self):
        self.startup_df = pd.read_csv("../output/results v3/df_with_examples.csv")
        cols = ['organization name', 'founded date','website','description_all', 'industries_parsed', 'generated_description', 'parsed_description', 'Tasks/Jobs', 'Industry', 'Customers','generated_description_conf_interval','parsed_description_conf_interval']
        self.df = self.startup_df[cols]
        self.df = self.df.merge(self.onet_df,left_on="organization name",right_on="organization_name")
        self.df = self.df.merge(self.codes,left_on="onet_title",right_on="Title")

    def getWage(self):
        self.wage_employment = pd.read_csv("../input/wage_employment/wage_employment_bls.csv")
        self.wage_employment = self.wage_employment[self.wage_employment.O_GROUP == "detailed"]
        self.wage_employment = self.wage_employment.rename({"OCC_CODE":"Detailed Occupation"},axis=1)
        self.wage_employment = self.wage_employment.replace("#",np.nan).replace("*",np.nan).replace({",":""}, regex=True)
        self.wage_employment[["H_MEDIAN","A_MEDIAN"]] = self.wage_employment[["H_MEDIAN","A_MEDIAN"]].astype("float")
        self.wage_employment["H_MEDIAN"] = self.wage_employment["H_MEDIAN"].round(0)
        
    def getSkills(self):
        self.skills_df = pd.read_csv("../input/skills/Skills.csv")
        self.skills_df = self.skills_df[self.skills_df["Scale Name"]=="Importance"]
        self.skills_df = self.skills_df.rename(columns={"O*NET-SOC Code":"Detailed Occupation"})
        self.skills_df["Detailed Occupation"] = self.skills_df["Detailed Occupation"].replace(r'\.\d{2}', '', regex=True)
        self.skills_df = self.skills_df.pivot_table(index=self.skills_df["Detailed Occupation"], columns='Element Name', values='Data Value', fill_value=0)
    
    def getSkillsSOC(self):
        self.skills_df = pd.read_csv("../input/skills_df_soc_codes.csv",index_col=0)
        
    def getWageSkills(self):
        self.getWage()
        self.getSkillsSOC()
        self.bls_df = self.skills_df.merge(self.wage_employment, on="Detailed Occupation", how="inner")
        
        
    def BLS(self):
        self.titles_df = self.getONETTitles()
        self.bls_df = self.titles_df.merge(self.bls_df, on="Detailed Occupation", how="right")

        
                

        # plt.figure(figsize=(14, 6))
        # for major_group_name in self.bls_df['Major Group Name'].unique():
        #     subset = self.bls_df[self.bls_df['Major Group Name'] == major_group_name]
        #     plt.scatter(subset['onet_weight'], subset['H_MEDIAN'], label=major_group_name)
        # plt.xlabel('ONET Weight')
        # plt.ylabel('Hourly Median Wage')
        # plt.title('Hourly Median Wage vs. ONET Weight')
        # plt.legend(loc='best', bbox_to_anchor=(1.05, 1), fontsize='small')
        # plt.tight_layout()
        # plt.show()
    
        # # Scatter plot for A_MEDIAN (Annual Median Wage)
        # plt.figure(figsize=(14, 6))
        # for major_group_name in self.bls_df['Major Group Name'].unique():
        #     subset = self.bls_df[self.bls_df['Major Group Name'] == major_group_name]
        #     plt.scatter(subset['onet_weight'], subset['A_MEDIAN'], label=major_group_name)
        # plt.xlabel('ONET Weight')
        # plt.ylabel('Annual Median Wage')
        # plt.title('Annual Median Wage vs. ONET Weight')
        # plt.legend(loc='best', bbox_to_anchor=(1.05, 1), fontsize='small')
        # plt.tight_layout()
        # plt.show()


   

In [16]:
out = output()
out.getONET(task_sim=.67,title_sim=.7)
out.getWageSkills()
out.BLS()
# out.printExamples()
# out.generateOutput()
# out.writeExamples()

1683 tasks with task similarity > 0.67 and title similarity > 0.7
851 startups out of 2188 decomposed


In [17]:
df = pd.read_csv("../output/results v3/onet_df.csv")
df = df[df.task_similarity > .67]
len(df["organization_name"].unique())

1827

In [448]:
wage_employment = out.wage_employment
skills_df = pd.read_csv("../input/skills_df_soc_codes.csv")


In [462]:
bls_df = out.bls_df


In [419]:
out.titles_df

,onet_title,onet_weight,Detailed Occupation
78,Market Research Analysts and Marketing Special...,54.97,13-1161
29,Data Scientists,29.19,15-2051
27,Customer Service Representatives,27.92,43-4051
53,Financial and Investment Analysts,24.50,13-2051
79,Marketing Managers,12.90,11-2021
...,...,...,...
77,Logisticians,0.22,13-1081
81,Mechanical Engineers,0.17,17-2141
150,"Wholesale and Retail Buyers, Except Farm Products",0.17,13-1022
16,Chief Executives,0.14,11-1011


In [421]:
out.titles_df.merge(out.bls_df, on="Detailed Occupation", how="right")

,onet_title,onet_weight,Detailed Occupation,Unnamed: 0,broad_occ,Major Group,Minor Group,Broad Group,Title,Major Group Name,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,Facilities Managers,0.50,11-3013,0.0,11-301,11-0000,11-3000,11-3010,Facilities Managers,Management Occupations,...,49.20,64.42,81.26,"60,840","78,130","102,340","133,990","169,020",NaN,NaN
1,NaN,NaN,11-9039,1.0,11-903,11-0000,11-9000,11-9030,"Education Administrators, All Other",Management Occupations,...,42.53,57.75,76.06,"49,470","64,800","88,460","120,120","158,200",NaN,NaN
2,NaN,NaN,11-9072,2.0,11-907,11-0000,11-9000,11-9070,"Entertainment and Recreation Managers, Except ...",Management Occupations,...,35.32,47.68,64.27,"42,730","53,480","73,460","99,170","133,680",NaN,NaN
3,Project Management Specialists,2.39,13-1082,3.0,13-108,13-0000,13-1000,13-1080,Project Management Specialists,Business and Financial Operations Occupations,...,47.39,62.35,78.39,"57,500","74,100","98,580","129,690","163,040",NaN,NaN
4,NaN,NaN,13-2022,4.0,13-202,13-0000,13-2000,13-2020,Appraisers of Personal and Business Property,Business and Financial Operations Occupations,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,NaN,NaN,53-1047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29.09,36.51,44.30,"39,440","47,560","60,510","75,940","92,140",NaN,NaN
856,NaN,NaN,53-3099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.16,22.33,30.49,"24,550","28,890","35,690","46,440","63,410",NaN,NaN
857,NaN,NaN,53-4099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21.47,26.12,41.10,"35,420","39,330","44,660","54,320","85,490",NaN,NaN
858,NaN,NaN,53-6099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.37,25.00,30.62,"27,340","31,700","40,300","52,000","63,680",NaN,NaN


In [463]:
bls_df = bls_df.replace(np.nan,0)

In [464]:
bls_df = bls_df[(bls_df["onet_weight"]<100) & (bls_df["onet_weight"]>3)]

In [1]:
plt.figure(figsize=(14, 10))
scatter = plt.scatter(x=bls_df['onet_weight'], y=bls_df['H_MEDIAN'], c=bls_df['Critical Thinking'], cmap='viridis', alpha=0.7)
plt.xlabel('ONET WEIGHT')
plt.ylabel('Hourly Median Wage (H_MEDIAN)')
plt.title('Writing vs Hourly Median Wage with ONET Weight as Color')
plt.colorbar(scatter, label='Critical Thinking')
plt.show()

NameError: name 'plt' is not defined

/var/folders/nf/2dvvlnxj5v962kwwzmpfj17h0000gn/T/ipykernel_54128/2310227502.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bls_df["H_MEDIAN"] = bls_df["H_MEDIAN"].round(0)


In [250]:
bls_df[bls_df["onet_weight"]>20]

,onet_title,onet_weight,Detailed Occupation,Active Learning,Active Listening,Complex Problem Solving,Coordination,Critical Thinking,Equipment Maintenance,Equipment Selection,...,A_PCT90,ANNUAL,HOURLY,Major Group,Minor Group,Broad Group,Title,Major Group Name,Minor Group Name,Broad Group Name
0,Market Research Analysts and Marketing Special...,54.97,13-1161,3.625,3.815,3.88,3.125,3.875,1.00,1.000,...,137040,0,0,13-0000,13-1000,13-1160,Market Research Analysts and Marketing Special...,Business and Financial Operations Occupations,Business Operations Specialists,Market Research Analysts and Marketing Special...
1,Data Scientists,29.19,15-2051,3.685,3.880,3.50,3.190,4.000,1.06,1.565,...,184090,0,0,15-0000,15-2000,15-2050,Data Scientists,Computer and Mathematical Occupations,Mathematical Science Occupations,Data Scientists
2,Customer Service Representatives,27.92,43-4051,2.880,4.000,3.12,2.880,3.250,1.00,1.000,...,61250,0,0,43-0000,43-4000,43-4050,Customer Service Representatives,Office and Administrative Support Occupations,Information and Record Clerks,Customer Service Representatives
3,Financial and Investment Analysts,24.50,13-2051,0.000,0.000,0.00,0.000,0.000,0.00,0.000,...,175840,0,0,13-0000,13-2000,13-2050,Financial and Investment Analysts,Business and Financial Operations Occupations,Financial Specialists,Financial Analysts and Advisors


In [251]:
bls_df

,onet_title,onet_weight,Detailed Occupation,Active Learning,Active Listening,Complex Problem Solving,Coordination,Critical Thinking,Equipment Maintenance,Equipment Selection,...,A_PCT90,ANNUAL,HOURLY,Major Group,Minor Group,Broad Group,Title,Major Group Name,Minor Group Name,Broad Group Name
0,Market Research Analysts and Marketing Special...,54.97,13-1161,3.625,3.815,3.88,3.125,3.875,1.00,1.000,...,137040,0,0,13-0000,13-1000,13-1160,Market Research Analysts and Marketing Special...,Business and Financial Operations Occupations,Business Operations Specialists,Market Research Analysts and Marketing Special...
1,Data Scientists,29.19,15-2051,3.685,3.880,3.50,3.190,4.000,1.06,1.565,...,184090,0,0,15-0000,15-2000,15-2050,Data Scientists,Computer and Mathematical Occupations,Mathematical Science Occupations,Data Scientists
2,Customer Service Representatives,27.92,43-4051,2.880,4.000,3.12,2.880,3.250,1.00,1.000,...,61250,0,0,43-0000,43-4000,43-4050,Customer Service Representatives,Office and Administrative Support Occupations,Information and Record Clerks,Customer Service Representatives
3,Financial and Investment Analysts,24.50,13-2051,0.000,0.000,0.00,0.000,0.000,0.00,0.000,...,175840,0,0,13-0000,13-2000,13-2050,Financial and Investment Analysts,Business and Financial Operations Occupations,Financial Specialists,Financial Analysts and Advisors
4,Marketing Managers,12.90,11-2021,3.880,3.880,3.62,3.500,3.880,1.00,1.000,...,0,0,0,11-0000,11-2000,11-2020,Marketing Managers,Management Occupations,Advertising Marketing Promotions Public Relati...,Marketing and Sales Managers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,0,0.00,53-3099,0.000,0.000,0.00,0.000,0.000,0.00,0.000,...,63410,0,0,53-0000,53-3000,53-3090,Motor Vehicle Operators All Other,Transportation and Material Moving Occupations,Motor Vehicle Operators,Miscellaneous Motor Vehicle Operators
835,0,0.00,53-4099,0.000,0.000,0.00,0.000,0.000,0.00,0.000,...,85490,0,0,53-0000,53-4000,53-4090,Rail Transportation Workers All Other,Transportation and Material Moving Occupations,Rail Transportation Workers,Miscellaneous Rail Transportation Workers
836,0,0.00,53-6032,0.000,0.000,0.00,0.000,0.000,0.00,0.000,...,53820,0,0,53-0000,53-6000,53-6030,Aircraft Service Attendants,Transportation and Material Moving Occupations,Other Transportation Workers,Transportation Service Attendants
837,0,0.00,53-6099,0.000,0.000,0.00,0.000,0.000,0.00,0.000,...,63680,0,0,53-0000,53-6000,53-6090,Transportation Workers All Other,Transportation and Material Moving Occupations,Other Transportation Workers,Miscellaneous Transportation Workers


In [233]:
out.skills_df.loc["43-4051"]

Element Name
Active Learning                      2.88
Active Listening                     4.00
Complex Problem Solving              3.12
Coordination                         2.88
Critical Thinking                    3.25
Equipment Maintenance                1.00
Equipment Selection                  1.00
Installation                         1.00
Instructing                          2.75
Judgment and Decision Making         2.88
Learning Strategies                  2.62
Management of Financial Resources    1.38
Management of Material Resources     1.38
Management of Personnel Resources    2.38
Mathematics                          2.25
Monitoring                           3.00
Negotiation                          3.00
Operation and Control                1.00
Operations Analysis                  1.62
Operations Monitoring                1.75
Persuasion                           3.00
Programming                          1.50
Quality Control Analysis             1.12
Reading Comprehension

In [303]:
# df = pd.read_csv("../input/skills/Skills.csv")

# df[df.Title.str.contains("Investment")].Title.unique()
# df[df["O*NET-SOC Code"].str.startswith("13-205")]

,O*NET-SOC Code,Title,Element ID,Element Name,Scale ID,Scale Name,Data Value,N,Standard Error,Lower CI Bound,Upper CI Bound,Recommend Suppress,Not Relevant,Date,Domain Source
5880,13-2052.00,Personal Financial Advisors,2.A.1.a,Reading Comprehension,IM,Importance,4.00,8.0,0.0000,4.0000,4.0000,N,NaN,08/2020,Analyst
5881,13-2052.00,Personal Financial Advisors,2.A.1.a,Reading Comprehension,LV,Level,4.00,8.0,0.0000,4.0000,4.0000,N,N,08/2020,Analyst
5882,13-2052.00,Personal Financial Advisors,2.A.1.b,Active Listening,IM,Importance,4.00,8.0,0.0000,4.0000,4.0000,N,NaN,08/2020,Analyst
5883,13-2052.00,Personal Financial Advisors,2.A.1.b,Active Listening,LV,Level,4.00,8.0,0.0000,4.0000,4.0000,N,N,08/2020,Analyst
5884,13-2052.00,Personal Financial Advisors,2.A.1.c,Writing,IM,Importance,4.00,8.0,0.0000,4.0000,4.0000,N,NaN,08/2020,Analyst
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6015,13-2053.00,Insurance Underwriters,2.B.5.b,Management of Financial Resources,LV,Level,1.12,8.0,0.1250,0.8800,1.3700,N,N,08/2018,Analyst
6016,13-2053.00,Insurance Underwriters,2.B.5.c,Management of Material Resources,IM,Importance,2.00,8.0,0.0000,2.0000,2.0000,N,NaN,08/2018,Analyst
6017,13-2053.00,Insurance Underwriters,2.B.5.c,Management of Material Resources,LV,Level,1.00,8.0,0.0000,1.0000,1.0000,N,N,08/2018,Analyst
6018,13-2053.00,Insurance Underwriters,2.B.5.d,Management of Personnel Resources,IM,Importance,2.38,8.0,0.1830,2.0164,2.7336,N,NaN,08/2018,Analyst


In [377]:
skills_df = skills_df.rename({"O*NET-SOC Code":"Detailed Occupation"},axis=1)
skills_df.head()

Element Name,Active Learning,Active Listening,Complex Problem Solving,Coordination,Critical Thinking,Equipment Maintenance,Equipment Selection,Installation,Instructing,Judgment and Decision Making,...,Service Orientation,Social Perceptiveness,Speaking,Systems Analysis,Systems Evaluation,Technology Design,Time Management,Troubleshooting,Writing,broad_occ
0,3.75,4.00,4.19,4.00,4.25,1.0,1.12,1.0,3.315,4.315,...,3.185,4.00,4.125,4.00,4.065,1.815,3.69,1.25,4.12,11-101
1,3.62,4.00,3.62,3.88,3.88,1.0,1.00,1.0,3.000,3.620,...,3.250,3.75,4.000,3.12,3.120,1.500,3.62,1.75,3.50,11-102
2,3.25,4.12,3.50,3.50,4.00,1.0,1.12,1.0,2.880,3.750,...,3.120,4.00,4.000,3.12,3.120,1.750,3.50,1.00,3.75,11-201
3,3.88,3.88,3.62,3.50,3.88,1.0,1.00,1.0,3.000,3.750,...,3.120,3.88,3.880,3.25,3.500,1.750,3.50,1.00,3.25,11-202
4,3.75,4.00,3.75,3.88,3.88,1.0,1.00,1.0,3.620,3.750,...,3.880,3.88,4.000,3.62,3.620,1.750,3.50,1.00,3.62,11-202


In [378]:
skills_df = out.skills_df.reset_index()

In [379]:
# total = skills_df.merge(codes,on="Detailed Occupation", how="right")
# cols = codes.columns
# df_broad = total[total["Active Learning"].isnull()]
# df_broad["broad_occ"] = df_broad["Detailed Occupation"].apply(lambda x: x[:6])
# df_broad = df_broad[list(["broad_occ"]+list(cols))]
# skills_df["broad_occ"] = skills_df["Detailed Occupation"].apply(lambda x: x[:6])
# skills_df = skills_df.drop(columns=["Detailed Occupation"])
# df_broad = df_broad.merge(skills_df_broad,on="broad_occ",how="inner")
# total = total[~total["Active Learning"].isnull()]
# skills_df_soc_codes = pd.concat([df_broad,total],axis=0)
# skills_df_soc_codes.to_csv('../input/skills_df_soc_codes.csv')

/var/folders/nf/2dvvlnxj5v962kwwzmpfj17h0000gn/T/ipykernel_54128/2898062444.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_broad["broad_occ"] = df_broad["Detailed Occupation"].apply(lambda x: x[:6])


In [387]:
skills_df_soc_codes

,broad_occ,Major Group,Minor Group,Broad Group,Detailed Occupation,Title,Major Group Name,Minor Group Name,Broad Group Name,Active Learning,...,Science,Service Orientation,Social Perceptiveness,Speaking,Systems Analysis,Systems Evaluation,Technology Design,Time Management,Troubleshooting,Writing
0,11-301,11-0000,11-3000,11-3010,11-3013,Facilities Managers,Management Occupations,Operations Specialties Managers,Administrative Services and Facilities Managers,3.120000,...,1.000000,3.250000,3.250000,3.880000,3.12,2.750000,1.500000,4.000000,1.880000,3.750000
1,11-903,11-0000,11-9000,11-9030,11-9039,"Education Administrators, All Other",Management Occupations,Other Management Occupations,Education and Childcare Administrators,3.753333,...,1.583333,3.753333,3.960000,4.126667,3.25,3.416667,1.663333,3.876667,1.500000,3.956667
2,11-907,11-0000,11-9000,11-9070,11-9072,"Entertainment and Recreation Managers, Except ...",Management Occupations,Other Management Occupations,Entertainment and Recreation Managers,3.500000,...,1.620000,3.750000,3.750000,3.880000,3.12,3.120000,1.500000,3.750000,1.750000,3.620000
3,13-108,13-0000,13-1000,13-1080,13-1082,Project Management Specialists,Business and Financial Operations Occupations,Business Operations Specialists,Logisticians and Project Management Specialists,3.456667,...,2.000000,3.083333,3.246667,3.793333,3.79,3.626667,2.126667,3.460000,1.626667,3.586667
4,13-202,13-0000,13-2000,13-2020,13-2022,Appraisers of Personal and Business Property,Business and Financial Operations Occupations,Financial Specialists,Property Appraisers and Assessors,3.370000,...,1.690000,2.880000,3.190000,3.750000,2.69,2.370000,1.380000,3.120000,1.690000,3.440000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,NaN,53-0000,53-7000,53-7070,53-7071,Gas Compressor and Gas Pumping Station Operators,Transportation and Material Moving Occupations,Material Moving Workers,Pumping Station Operators,2.880000,...,1.880000,2.250000,2.750000,3.000000,2.62,2.000000,1.880000,3.000000,3.120000,3.000000
843,NaN,53-0000,53-7000,53-7070,53-7072,"Pump Operators, Except Wellhead Pumpers",Transportation and Material Moving Occupations,Material Moving Workers,Pumping Station Operators,2.880000,...,2.000000,2.380000,2.750000,3.120000,2.38,2.120000,1.880000,3.000000,3.000000,2.880000
844,NaN,53-0000,53-7000,53-7070,53-7073,Wellhead Pumpers,Transportation and Material Moving Occupations,Material Moving Workers,Pumping Station Operators,2.380000,...,1.120000,1.880000,2.500000,3.000000,2.00,2.000000,1.500000,2.750000,3.120000,2.620000
845,NaN,53-0000,53-7000,53-7080,53-7081,Refuse and Recyclable Material Collectors,Transportation and Material Moving Occupations,Material Moving Workers,Refuse and Recyclable Material Collectors,2.250000,...,1.000000,2.380000,2.500000,2.880000,1.38,1.380000,1.000000,2.500000,2.500000,2.500000


In [271]:
codes = pd.read_csv("../input/soc_codes/soc_codes.csv",index_col=0)
codes[codes.Title=="Financial and Investment Analysts"]

,Major Group,Minor Group,Broad Group,Detailed Occupation,Title,Major Group Name,Minor Group Name,Broad Group Name
120,13-0000,13-2000,13-2050,13-2051,Financial and Investment Analysts,Business and Financial Operations Occupations,Financial Specialists,Financial Analysts and Advisors


,Detailed Occupation,Minor Group,Broad Group,Title,Major Group Name,Minor Group Name,Broad Group Name,broad_occ,Active Learning,Active Listening,...,Science,Service Orientation,Social Perceptiveness,Speaking,Systems Analysis,Systems Evaluation,Technology Design,Time Management,Troubleshooting,Writing
0,11-1031,11-1000,11-1030,Legislators,Management Occupations,Top Executives,Legislators,11-103,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11-2032,11-2000,11-2030,Public Relations Managers,Management Occupations,"Advertising, Marketing, Promotions, Public Rel...",Public Relations and Fundraising Managers,11-203,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11-2033,11-2000,11-2030,Fundraising Managers,Management Occupations,"Advertising, Marketing, Promotions, Public Rel...",Public Relations and Fundraising Managers,11-203,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,19-1099,19-1000,19-1090,"Life Scientists, All Other","Life, Physical, and Social Science Occupations",Life Scientists,Miscellaneous Life Scientists,19-109,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,21-2099,21-2000,21-2090,"Religious Workers, All Other",Community and Social Service Occupations,Religious Workers,Miscellaneous Religious Workers,21-209,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,25-3031,25-3000,25-3030,"Substitute Teachers, Short-Term",Educational Instruction and Library Occupations,Other Teachers and Instructors,"Substitute Teachers, Short-Term",25-303,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,25-3099,25-3000,25-3090,"Teachers and Instructors, All Other",Educational Instruction and Library Occupations,Other Teachers and Instructors,Miscellaneous Teachers and Instructors,25-309,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,25-9099,25-9000,25-9090,"Educational Instruction and Library Workers, A...",Educational Instruction and Library Occupations,Other Educational Instruction and Library Occu...,Miscellaneous Educational Instruction and Libr...,25-909,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,27-2091,27-2000,27-2090,"Disc Jockeys, Except Radio","Arts, Design, Entertainment, Sports, and Media...","Entertainers and Performers, Sports and Relate...","Miscellaneous Entertainers and Performers, Spo...",27-209,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,27-2099,27-2000,27-2090,"Entertainers and Performers, Sports and Relate...","Arts, Design, Entertainment, Sports, and Media...","Entertainers and Performers, Sports and Relate...","Miscellaneous Entertainers and Performers, Spo...",27-209,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
